In [177]:
import math
import random
import pandas as pd
import numpy as np

In [178]:
df1 = pd.read_csv('TSPA.csv', header = None, sep = ';')
df2 = pd.read_csv('TSPB.csv', header = None, sep = ';')
df3 = pd.read_csv('TSPC.csv', header = None, sep = ';')
df4 = pd.read_csv('TSPD.csv', header = None, sep = ';')

In [179]:
df1

,0,1,2
0,1510,353,84
1,3042,123,483
2,2145,389,1462
3,1030,365,1986
4,2125,360,145
...,...,...,...
195,1023,1258,197
196,3192,1271,1183
197,3735,1879,1461
198,3983,299,1309


### Distance matrix
Node costs included

In [180]:
def calculate_distance(city1, city2):
    return math.ceil(math.sqrt((city1[0]-city2[0])**2 + (city1[1]-city2[1])**2))

In [181]:
def create_distance_matrix(df):
    distance_matrix = [[df.iloc[j, 2] for j in range(df.shape[0])] for i in range(df.shape[0])]
    for i in range(df.shape[0]):
        for j in range(df.shape[0]):
            if i == j:
                break
            dist = calculate_distance(df.values[i][:2], df.values[j][:2])
            distance_matrix[i][j] += dist
            distance_matrix[j][i] += dist
    return distance_matrix

### Helpful debugging function for checking the length of the cycle

In [182]:
def check_cycle_length(df, path):
    distance_matrix = create_distance_matrix(df)
    dist = 0
    for index, node in enumerate(path[:-1]):
        dist += distance_matrix[node][path[(index+1) % len(path)]]
    dist += distance_matrix[path[-1]][path[0]]
    return dist

### Random Solution

In [183]:
def random_TSP(cost_matrix):
    path = random.sample(range(0, len(cost_matrix)), math.ceil(len(cost_matrix) / 2))
    dist = 0
    for index, item in enumerate(path):
        dist += cost_matrix[item][path[(index+1) % len(path)]]
    return dist, path

### Nearest Neighbour

In [184]:
def nearest_TSP(cost_matrix, starting_city):
    dist = 0
    path = [starting_city]

    cycle_length = math.ceil(len(cost_matrix) / 2)

    for i in range(0, cycle_length - 1):
        shortest_distance = math.inf
        shortest_city = -1

        for j in range(len(cost_matrix)):
            if j in path:
                continue
            if cost_matrix[path[-1]][j] < shortest_distance:
                shortest_distance = cost_matrix[path[-1]][j]
                shortest_city = j
        
        path.append(shortest_city)
        dist += shortest_distance

    dist += cost_matrix[path[-1]][starting_city]

    return dist, path

### Greedy Cycle

In [185]:
def greedy_TSP(cost_matrix, starting_city):
    dist = 0
    path = [starting_city]

    cycle_length = math.ceil(len(cost_matrix) / 2)

    shortest_distance = math.inf
    shortest_city = -1

    for j in range(len(cost_matrix)):
        if j in path:
            continue
        if cost_matrix[path[-1]][j] < shortest_distance:
            shortest_distance = cost_matrix[path[-1]][j]
            shortest_city = j
    
    path.append(shortest_city)
    dist += shortest_distance
    dist += cost_matrix[path[-1]][starting_city]

    for i in range(0, cycle_length - 2):
        shortest_extra_distance = math.inf
        shortest_city = -1
        shortest_city_index = -1

        for j in range(len(cost_matrix)):
            if j in path:
                continue
            
            for k in range(len(path)):
                extra_distance = cost_matrix[path[k]][j] + cost_matrix[j][path[(k+1) % len(path)]] - cost_matrix[path[k]][path[(k+1) % len(path)]]
                if extra_distance < shortest_extra_distance:
                    shortest_extra_distance = extra_distance
                    shortest_city = j
                    shortest_city_index = k+1
            
        
        path.insert(shortest_city_index, shortest_city)
        dist += shortest_extra_distance
    


    return dist, path

### Compare methods

In [186]:
iterations = 200 ### change hardcode to matrix size
nearest_bests = []
greedy_bests = []

for id, df in enumerate([df1, df2, df3, df4]):
    distance_matrix = create_distance_matrix(df)

    print(f'\n==============[Dataset {id+1}]==============')
    print(f'RANDOM TSP')

    min_dist = math.inf
    max_dist = 0
    avg_dist = 0
    best_path = []

    for i in range(0, iterations):
        dist, path = random_TSP(distance_matrix)
        avg_dist += dist

        if dist < min_dist:
            min_dist = dist
            best_path = path
        elif dist > max_dist:
            max_dist = dist
    
    avg_dist /= iterations

    print(f'Best solution: {best_path}')
    print(f'Shortest distance: {min_dist}')
    print(f'Longest distance: {max_dist}')
    print(f'Average distance: {avg_dist}')

    print(f'\nNEAREST NEIGHBOUR TSP')

    min_dist = math.inf
    max_dist = 0
    avg_dist = 0
    best_path = []

    for i in range(0, iterations):
        dist, path = nearest_TSP(distance_matrix, i)
        avg_dist += dist

        if dist < min_dist:
            min_dist = dist
            best_path = path
        elif dist > max_dist:
            max_dist = dist
    
    avg_dist /= iterations

    assert check_cycle_length(df, best_path) == min_dist

    nearest_bests.append(min_dist)

    print(f'Best solution: {best_path}')
    print(f'Shortest distance: {min_dist}')
    print(f'Longest distance: {max_dist}')
    print(f'Average distance: {avg_dist}')

    print(f'\nGREEDY CYCLE TSP')

    min_dist = math.inf
    max_dist = 0
    avg_dist = 0
    best_path = []

    for i in range(0, iterations):
        dist, path = greedy_TSP(distance_matrix, i)
        avg_dist += dist

        if dist < min_dist:
            min_dist = dist
            best_path = path
        elif dist > max_dist:
            max_dist = dist
    
    avg_dist /= iterations

    assert check_cycle_length(df, best_path) == min_dist

    greedy_bests.append(min_dist)

    print(f'Best solution: {best_path}')
    print(f'Shortest distance: {min_dist}')
    print(f'Longest distance: {max_dist}')
    print(f'Average distance: {avg_dist}')


==============[Dataset 1]==============
RANDOM TSP
Best solution: [79, 112, 197, 2, 1, 181, 180, 32, 31, 59, 168, 167, 13, 111, 28, 194, 171, 147, 103, 55, 184, 145, 102, 66, 124, 99, 140, 93, 131, 113, 178, 96, 117, 50, 18, 176, 98, 177, 119, 48, 190, 173, 34, 163, 77, 82, 24, 4, 104, 150, 138, 120, 196, 198, 8, 42, 88, 175, 10, 142, 107, 9, 51, 70, 137, 155, 164, 148, 83, 116, 33, 156, 76, 143, 161, 67, 183, 86, 57, 195, 91, 58, 6, 130, 92, 144, 49, 135, 12, 52, 43, 192, 41, 97, 126, 108, 7, 11, 186, 141]
Shortest distance: 238152
Longest distance: 297430
Average distance: 265165.175

NEAREST NEIGHBOUR TSP
Best solution: [1, 177, 75, 189, 119, 48, 106, 11, 152, 130, 26, 8, 95, 169, 135, 167, 186, 79, 141, 6, 98, 66, 72, 94, 31, 80, 14, 111, 73, 112, 51, 190, 156, 172, 24, 87, 154, 81, 117, 55, 195, 74, 163, 113, 185, 96, 147, 59, 143, 178, 19, 0, 149, 114, 4, 77, 43, 50, 121, 91, 76, 22, 53, 62, 108, 171, 21, 194, 144, 102, 133, 180, 32, 132, 128, 164, 159, 27, 116, 64, 20, 71, 183,

In [189]:
for i in range(4):
    print(f"df{i+1} Nearest {nearest_bests[i]}")

df1 Nearest 84524
df2 Nearest 77602
df3 Nearest 56352
df4 Nearest 50393


In [190]:
for i in range(4):
    print(f"df{i+1} Greedy {greedy_bests[i]}")

df1 Greedy 75726
df2 Greedy 68803
df3 Greedy 53527
df4 Greedy 50288
